####

#### ♣♣♣ 다중분류를 위한 OvO ♣♣♣
- OvO(One-vs-One) 즉, 1:1 방식 2진 분류기 구성

[1] 모듈로딩 및 데이터 준비<hr>

In [15]:
## [1-1] 모듈로딩
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix


In [16]:
## [1-2] 데이터 로딩
iris = load_iris()

X, y = iris.data, iris.target

print(f"클래스: {iris.target_names.tolist()}  (라벨: {sorted(np.unique(y))})")


클래스: ['setosa', 'versicolor', 'virginica']  (라벨: [0, 1, 2])


[2] 학습 준비 <hr>

In [24]:
## [2-1] 학습/테스트 분할 (층화)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [25]:
## [2-2] 스케일링 (수동)
scaler = StandardScaler()
scaler.fit(X_train)

X_train_sc = scaler.transform(X_train)
X_test_sc  = scaler.transform(X_test)

[3] 학습진행<HR>

In [ ]:
## [3-1] OVR => 기본값 : 가장 높은 점수의 클래스로 예측
# 모델1 (0 vs rest): 0일 확률 0.8
# 모델2 (1 vs rest): 1일 확률 0.2，
# 모델3 (2 vs rest): 2일 확률 0.3
# → 가장 높은 0.8 → 클래스 0 예측

## 모델 인스턴스 생성
ovrModel = SVC(kernel="rbf",
              C=1.0,
              gamma="scale",
              decision_function_shape='ovr',
              random_state=42)

## 학습 진행
ovrModel.fit(X_train_sc, y_train)

## 예측
y_pred = ovrModel.predict(X_test_sc)

## 모델 성능평가 
## OVR => 학습 데이터셋이 불균형데이터셋 => 클래스별로 성능지표 계산 진행 
##        macro 설정 
acc = accuracy_score(y_test, y_pred)
f1m = f1_score(y_test, y_pred, average="macro")
cm  = confusion_matrix(y_test, y_pred)

print(f"\n==== SVC - OVR ====")
print(f"Accuracy : {acc:.4f}")
print(f"Macro F1 : {f1m:.4f}")
print("Confusion Matrix:\n", cm)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))



==== SVC - OVR ====
Accuracy : 0.9667
Macro F1 : 0.9666
Confusion Matrix:
 [[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]

Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.90      0.95        10
   virginica       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



In [ ]:
## [3-2] OVO => 다수결 투표로 예측
# 모델1 (0 vs 1): 예측 = 0
# 모델2 (0 vs 2): 예측 = 2
# 모델3 (1 vs 2): 예측 = 2
# → 투표 결과: 0(1표), 1(0표), 2(2표) → 클래스 2 예측

## 모델 인스턴스 생성
ovoModel = SVC(kernel="rbf",
              C=1.0,
              gamma="scale",
              decision_function_shape='ovo',
              random_state=42)

## 학습 진행
ovoModel.fit(X_train_sc, y_train)

## 예측
y_pred = ovoModel.predict(X_test_sc)

## 모델 성능평가 
## OVO => 학습 데이터셋이 불균형데이터셋 => 클래스별로 성능지표 계산 진행 
##        macro 설정 
acc = accuracy_score(y_test, y_pred)
f1m = f1_score(y_test, y_pred, average='micro')
cm  = confusion_matrix(y_test, y_pred)


print(f"\n==== SVC - OVO ====")
print(f"Accuracy : {acc:.4f}")
print(f"Macro F1 : {f1m:.4f}")
print("Confusion Matrix:\n", cm)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=iris.target_names))



==== SVC - OVO ====
Accuracy : 0.9667
Macro F1 : 0.9667
Confusion Matrix:
 [[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]

Classification Report:
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.90      0.95        10
   virginica       0.91      1.00      0.95        10

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

